In [1]:
import numpy as np
import os,sys

sys.path.append('.')
sys.path.append('../RL_lib/Utils')
%load_ext autoreload
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os
print(os.getcwd())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/briangaudet/Study/Subjects/MachineLearning/Projects/PCM/PCM_v4/Imaging


In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

### For homogonous coords, the rotation matrix will act on the translation in the previous rotation matrix

In [7]:
import attitude_utils as attu
import optics_utils as optu
ap = attu.Euler_attitude()
def transform1(object_locations, agent_location, agent_q, C_cb, r_cb, ap):
    object_locations = object_locations - agent_location

    C_bn = ap.q2dcm(agent_q)

    # rotate to body-centered body frame then translate to camera centered
    bf_coords = object_locations.dot(C_bn.T) - r_cb

    # rotate to camera-centered camera frame
    cf_coords = bf_coords.dot(C_cb.T)
    return cf_coords

def transform2(object_locations, agent_location, agent_q, M_cb, r_cb, ap):
    #print(object_locations.shape)
    C_bn = ap.q2dcm(agent_q)
    M_bn = optu.ct2m(C_bn, agent_location)
    M = M_cb.dot(M_bn)
    object_locations = np.hstack((object_locations,np.ones((object_locations.shape[0],1))))
    cf_coords = object_locations.dot(M.T)
    cf_coords = cf_coords[:,0:3]
    return cf_coords

def transform3(object_locations, agent_location, agent_q, M_cb, r_cb, ap):
    #print(object_locations.shape)
    object_locations = object_locations - agent_location
    C_bn = ap.q2dcm(agent_q)
    M_bn = optu.ct2m(C_bn, r_cb)
    M = M_cb.dot(M_bn)
    object_locations = np.hstack((object_locations,np.ones((object_locations.shape[0],1))))
    cf_coords = object_locations.dot(M.T)
    cf_coords = cf_coords[:,0:3]
    return cf_coords

object_locations = np.asarray([200,10,-10])
agent_location= np.asarray([500,-20,15,])
yaw = 0.0 #+np.pi/16
pitch = 0.0
roll = 0.0
agent_q = np.asarray([yaw,pitch,roll])

yaw = 0.0 #+np.pi/16
pitch = 0.0
roll = np.pi
q = np.asarray([yaw,pitch,roll])
C_cb = ap.q2dcm(q)
r_cb = np.asarray([2,-3,1])
M_cb = optu.ct2m(C_cb,r_cb)
M_cb2 = optu.ct2m(C_cb,np.zeros(3))
print(M_cb)
c1 = transform1(object_locations, agent_location, agent_q, C_cb, r_cb, ap)
c2 = transform2(np.expand_dims(object_locations,axis=0), agent_location, agent_q, M_cb, r_cb, ap)
c3 = transform3(np.expand_dims(object_locations,axis=0), agent_location, agent_q, M_cb2, r_cb, ap)
print(c1)
print(c2)
print(c3)

Euler321 Attitude
[[ 1.0000000e+00  0.0000000e+00 -0.0000000e+00 -2.0000000e+00]
 [ 0.0000000e+00 -1.0000000e+00  1.2246468e-16  3.0000000e+00]
 [ 0.0000000e+00 -1.2246468e-16 -1.0000000e+00 -1.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
[-302.  -33.   26.]
[[-302.  -27.   24.]]
[[-302.  -33.   26.]]
